<a href="https://colab.research.google.com/github/seq2193/so-vits-svc-colab/blob/main/sovits3_0_%E6%8E%A8%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

初始项目：https://github.com/innnky/so-vits-svc

修改日期：2023年3月17日

In [ ]:
#@title #配置环境 & 安装依赖
#@markdown ###选择要克隆的github仓库分支
Clone = "32khz" #@param ["32khz", "48khz"]

if Clone == "32khz":
  !git clone https://github.com/svc-develop-team/so-vits-svc -b 3.0-32k
  print('已选择32k')
else:
  !git clone https://github.com/svc-develop-team/so-vits-svc -b 3.0-48k
  print('已选择48k')

%cd /content/so-vits-svc
!pip install -r requirements.txt
!pip install pyworld praat-parselmouth scikit-maad gradio
!pip install --upgrade gdown
#下载必要模型文件
!wget -P hubert/ https://github.com/bshall/hubert/releases/download/v0.1/hubert-soft-0d54a1f4.pt 

## 加载歌手模型

In [ ]:
#@title 加载Google云端硬盘
#@markdown 加载Google云端硬盘
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title 复制Google云端硬盘模型
#@markdown 若使用，自行更改文件夹名、模型名
model_path = '/content/drive/MyDrive/32k/G_22000.pth' #@param {type:"string"}
config_path = "/content/drive/MyDrive/32k/config.json" #@param {type:"string"}

!cp {model_path} /content/so-vits-svc/logs/44k
!cp {config_path} /content/so-vits-svc/configs

In [ ]:
#@title 外链其他 gdown 模型
#训练中歌手模型 "spk:teto" 962條 utau英日文声库
!gdown xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx  -O /content/so-vits-svc/logs/32k/G_31000.pth
!gdown xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx  -O /content/so-vits-svc/configs/config.json

In [ ]:
#@title 从huggingface下载模型
#@markdown ##选择你克隆的github仓库的分支和你最开始训练的保持一致
Clone = "32k" #@param ["32k","48k"]
pth_link="https://huggingface.co/Itoifi/so-vits-svc-acg-models/resolve/main/32k-tsubaki/G_20000.pth" #@param{type:"string"}
config_link="https://huggingface.co/Itoifi/so-vits-svc-acg-models/resolve/main/32k-tsubaki/config.json" #@param{type:"string"}
!wget -P /content/so-vits-svc/logs/{Clone} {pth_link}
!wget -O /content/so-vits-svc/configs/config.json  {config_link}

# 声音转换

In [ ]:
#@title 上传.wav音频文件 
#@markdown 
import os
from google.colab import files

save_dir = "/content/so-vits-svc/raw"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

uploaded = files.upload()
file_name = list(uploaded.keys())[0]
file_path = os.path.join(save_dir, file_name)
with open(file_path, "wb") as f:
    f.write(uploaded[file_name])
print(f"File saved at: {file_path}")

In [ ]:
#@markdown 下载示例音频合成 風になる
!gdown --id 14-OrWnNm6JBB6tJ7NV55N6T4kNyilDVD -O "/content/so-vits-svc/raw/風になる-短.wav"

In [ ]:
#@title 从Dropbox下载Vocal WAV
#@markdown Share_Link 中 www 改成dl 并且.wav后面删除 ?dl=0
wav_link="https://dl.dropbox.com/s/mh5yfihdmk1cnri/1_07.%20%E3%83%A4%E3%82%AD%E3%83%A2%E3%83%81_%28Vocals%29.wav" #@param{type:"string"}

!wget -P /content/so-vits-svc/raw {wav_link}

# 推理

In [ ]:
#@title 一键推理 inference_main.py 配置文件并合成
#@markdown 17行更改model_path为你自己训练的最新模型记录点 例如：NyaruTaffy.pth
model_path = "logs/32k/G_22000.pth" #@param {type:"string"}
#@markdown 23行clean_names 写待转换的音频名称，不含扩展名
clean_names = "\u30E4\u30AD\u30E2\u30C1_(Vocals)" #@param {type:"string"}
#@markdown 24行trans 填写变调半音数量 
trans = "0" #@param {type:"string"}
#@markdown 25行spk_list 填写合成的说话人名称 例如：猫雷 nyaru，塔菲 taffy
spk_list = "teto" #@param {type:"string"}
#@markdown 26行默认-40，嘈杂的音频可以-30，干声保留呼吸可以-50
slice_db = "-40" #@param {type:"string"}
#@markdown 27行音频输出格式:默认flac 或 wav
wav_format = "wav" #@param {type:"string"}


!sed -i -r 's|(model_path =)(\s+)(.+)|\1\2"{model_path}"|g' /content/so-vits-svc/inference_main.py
!sed -i -r 's|(clean_names =)(\s+)(.+)|\1\2["{clean_names}"]|g' /content/so-vits-svc/inference_main.py
!sed -i -r 's|(trans =)(\s+)(.+)|\1\2[{trans}]|g' /content/so-vits-svc/inference_main.py
!sed -i -r 's|(spk_list =)(\s+)(.+)|\1\2["{spk_list}"]|g' /content/so-vits-svc/inference_main.py
!sed -i -r 's|(slice_db =)(\s+)(.+)|\1\2{slice_db}|g' /content/so-vits-svc/inference_main.py
!sed -i -r 's|(wav_format =)(\s+)(.+)|\1\2"{wav_format}"|g' /content/so-vits-svc/inference_main.py
#启动推理
!python /content/so-vits-svc/inference_main.py

In [ ]:
#手动修改 inference_main.py
from google.colab import files
files.view('inference_main.py')

<IPython.core.display.Javascript object>

In [ ]:
#手动合成
!python inference_main.py

# Gradio 推理（WebUI）

In [ ]:
#@markdown 模型更名为model.pth，配置文件更名为config.json，并放置到checkpoints/spk 文件夹下

#@markdown spk_list 说话人名称
spk_list = "teto" #@param {type:"string"}
pth_name = "G_22000.pth" #@param {type:"string"}
%cd /content/so-vits-svc
!mkdir -p checkpoints/{spk_list}

!cp /content/so-vits-svc/logs/32k/{pth_name}  /content/so-vits-svc/checkpoints/{spk_list}
!cp /content/so-vits-svc/configs/config.json  /content/so-vits-svc/checkpoints/{spk_list}
!mv /content/so-vits-svc/checkpoints/{spk_list}/{pth_name}    /content/so-vits-svc/checkpoints/{spk_list}/model.pth

In [ ]:
#第47行修改才可以在colab使用 grVits.Vits.launch(share=True)
!python sovits_gradio.py

In [ ]:
pip install olefile

In [ ]:
#此程序码内容为 sovits_gradio.py
from inference.infer_tool_grad import VitsSvc
import gradio as gr
import os

class VitsGradio:
    def __init__(self):
        self.so = VitsSvc()
        self.lspk = []
        self.modelPaths = []
        for root,dirs,files in os.walk("checkpoints"):
            for dir in dirs:
                self.modelPaths.append(dir)
        with gr.Blocks() as self.Vits:
            with gr.Tab("VoiceConversion"):
                with gr.Row(visible=False) as self.VoiceConversion:
                    with gr.Column():
                        with gr.Row():
                            with gr.Column():
                                self.srcaudio = gr.Audio(label = "输入音频")
                                self.btnVC = gr.Button("说话人转换")
                            with gr.Column():
                                self.dsid = gr.Dropdown(label = "目标角色", choices = self.lspk)
                                self.tran = gr.Slider(label = "升降调", maximum = 60, minimum = -60, step = 1, value = 0)
                                self.th = gr.Slider(label = "切片阈值", maximum = 32767, minimum = -32768, step = 0.1, value = -40)
                        with gr.Row():
                            self.VCOutputs = gr.Audio()
                self.btnVC.click(self.so.inference, inputs=[self.srcaudio,self.dsid,self.tran,self.th], outputs=[self.VCOutputs])
            with gr.Tab("SelectModel"):
                with gr.Column():
                    modelstrs = gr.Dropdown(label = "模型", choices = self.modelPaths, value = self.modelPaths[0], type = "value")
                    devicestrs = gr.Dropdown(label = "设备", choices = ["cpu","cuda"], value = "cpu", type = "value")
                    btnMod = gr.Button("载入模型")
                    btnMod.click(self.loadModel, inputs=[modelstrs,devicestrs], outputs = [self.dsid,self.VoiceConversion])

    def loadModel(self, path, device):
        self.lspk = []
        self.so.set_device(device)
        self.so.loadCheckpoint(path)
        for spk, sid in self.so.hps.spk.items():
            self.lspk.append(spk)
        VChange = gr.update(visible = True)
        SDChange = gr.update(choices = self.lspk, value = self.lspk[0])
        return [SDChange,VChange]

grVits = VitsGradio()

grVits.Vits.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

# Onnx导出

In [ ]:
#安装 onnx 依赖
!pip install onnx onnxsim onnxruntime onnxoptimizer
!git pull

In [ ]:
#@markdown 模型更名为model.pth，配置文件更名为config.json，并放置到checkpoints/spk 文件夹下

#@markdown spk_list 说话人名称
spk_list = "teto" #@param {type:"string"}
pth_name = "G_22000.pth" #@param {type:"string"}
%cd /content/so-vits-svc
!mkdir -p checkpoints/{spk_list}

!cp /content/so-vits-svc/logs/32k/{pth_name}  /content/so-vits-svc/checkpoints/{spk_list}
!cp /content/so-vits-svc/configs/config.json  /content/so-vits-svc/checkpoints/{spk_list}
!mv /content/so-vits-svc/checkpoints/{spk_list}/{pth_name}    /content/so-vits-svc/checkpoints/{spk_list}/model.pth

In [ ]:
#修改 onnx_export.py 文件第15行 path
from google.colab import files
files.view('onnx_export.py')

<IPython.core.display.Javascript object>

In [ ]:
#@title 导出Onnx模型
#运行 onnx_export.py
#等待执行完毕，在你的项目文件夹下会生成一个model.onnx，即为导出的模型
!python onnx_export.py

# 附录

In [ ]:
#@title 附录：flac to wav 
#方法1 修改输入前后文件名称
%cd /content/so-vits-svc/raw
import subprocess
subprocess.call(['ffmpeg', '-i', 'XJ.ogg',
                   'vocals.wav'])
%cd /content/so-vits-svc

In [ ]:
#@title raw目录音频切片器（遇到CUDA错误时需要切片再推理）
!pip install pydub
import os
from pydub import AudioSegment

# 定义输入和输出目录
input_dir = "/content/diff-svc/raw"
output_dir = "/content/diff-svc/raw"

# 遍历输入目录中的所有音频文件
for file_name in os.listdir(input_dir):
    # 仅处理扩展名为 wav 的文件
    if file_name.endswith(".wav"):
        # 加载音频文件
        audio = AudioSegment.from_file(os.path.join(input_dir, file_name), format="wav")
        
        # 计算每60秒的毫秒数
        segment_length_ms = 60 * 1000
        
        # 切割音频文件
        segments = []
        for start_time in range(0, len(audio), segment_length_ms):
            end_time = start_time + segment_length_ms
            segment = audio[start_time:end_time]
            segments.append(segment)
        
        # 保存切割后的音频文件到输出目录
        for i, segment in enumerate(segments):
            output_file_name = f"{os.path.splitext(file_name)[0]}_{i}.wav"
            output_path = os.path.join(output_dir, output_file_name)
            segment.export(output_path, format="wav")
